In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('./../../designer/script/')
sys.path.append('./../')
sys.path.append('./../../')

from design import Design
from film import FilmSimple
from spectrum import Spectrum
from optimizer.LM_gradient_descent import LM_optimize_d_simple
from optimizer.adam import adam_optimize
import copy
from utils.loss import calculate_RMS
import pickle
import designer
from analyze_utils.structure import plot_layer_thickness


inc_ang = 0.
wls = np.linspace(400, 1000, 500)

In [19]:

def make_random_film(N, pert_budget, seed):
    np.random.seed(seed)
    d = np.random.random(N)
    np.random.seed(seed * 13 % 7)
    np.random.seed(np.random.randint(0, 100))
    pert_idx = np.random.randint(0, d.shape[0])
    d *= 3000 / np.sum(d)
    d[pert_idx] += 100

    film = FilmSimple('SiO2', 'TiO2', 'Si', d_init=d)
    film.add_spec_param(0., wls)
    film_pert = copy.deepcopy(film)

    d = copy.deepcopy(d) # change reference of d to new
    d[pert_idx] += pert_budget
    film_pert.update_d(d)
    return film, film_pert

def make_pert(dim, budgets, rep_num, show=False):

    losses = []
    for rep in range(rep_num):
        if show:
            print(f'dim {dim}, rep {rep}')
        losses.append([])
        for budget in budgets:
            film, film_pert = make_random_film(dim, budget, rep)

            # plot_layer_thickness(film)
            # plot_layer_thickness(film_pert)

            # print(f'before: loss={calculate_RMS(film, film_pert)}')
            adam_optimize(film_pert, [film.get_spec()], 100, alpha=1)
            # print(f'after: loss={calculate_RMS(film, film_pert)}')
            losses[-1].append(calculate_RMS(film, film_pert))
    losses = np.array(losses)
    stds = np.std(losses, axis=0)
    losses = np.mean(losses, axis=0)
    return losses, stds

def make_pert_LM(dim, budgets, rep_num, show=False):

    losses = []
    for rep in range(rep_num):
        if show:
            print(f'dim {dim}, rep {rep}')
        losses.append([])
        for budget in budgets:
            film, film_pert = make_random_film(dim, budget, rep)

            # plot_layer_thickness(film)
            # plot_layer_thickness(film_pert)

            # print(f'before: loss={calculate_RMS(film, film_pert)}')
            LM_optimize_d_simple(film_pert, [film.get_spec()], 1e-10, 1000)
            # print(f'after: loss={calculate_RMS(film, film_pert)}')
            losses[-1].append(calculate_RMS(film, film_pert))
    losses = np.array(losses)
    stds = np.std(losses, axis=0)
    losses = np.mean(losses, axis=0)
    return losses, stds


In [20]:
# adam
loss_adam, std_adam = [], []
layer_numbers = [5, 10, 50, 60, 70, 80, 100]
budgets=np.linspace(-100, 100, 20)
for N in layer_numbers:
    cur_loss, cur_std = make_pert(N, budgets, rep_num=10, show=True)
    loss_adam.append(cur_loss)
    std_adam.append(cur_std)

dim 5, rep 0
dim 5, rep 1
dim 5, rep 2
dim 5, rep 3
dim 5, rep 4
dim 5, rep 5
dim 5, rep 6
dim 5, rep 7
dim 5, rep 8
dim 5, rep 9
dim 10, rep 0
dim 10, rep 1
dim 10, rep 2
dim 10, rep 3
dim 10, rep 4
dim 10, rep 5
dim 10, rep 6
dim 10, rep 7
dim 10, rep 8
dim 10, rep 9
dim 50, rep 0
dim 50, rep 1
dim 50, rep 2
dim 50, rep 3
dim 50, rep 4
dim 50, rep 5
dim 50, rep 6
dim 50, rep 7
dim 50, rep 8
dim 50, rep 9
dim 60, rep 0
dim 60, rep 1
dim 60, rep 2
dim 60, rep 3
dim 60, rep 4
dim 60, rep 5
dim 60, rep 6
dim 60, rep 7
dim 60, rep 8
dim 60, rep 9
dim 70, rep 0
dim 70, rep 1
dim 70, rep 2
dim 70, rep 3
dim 70, rep 4
dim 70, rep 5
dim 70, rep 6
dim 70, rep 7
dim 70, rep 8
dim 70, rep 9
dim 80, rep 0
dim 80, rep 1
dim 80, rep 2
dim 80, rep 3
dim 80, rep 4
dim 80, rep 5
dim 80, rep 6
dim 80, rep 7
dim 80, rep 8
dim 80, rep 9
dim 100, rep 0
dim 100, rep 1
dim 100, rep 2
dim 100, rep 3
dim 100, rep 4
dim 100, rep 5
dim 100, rep 6
dim 100, rep 7
dim 100, rep 8
dim 100, rep 9


In [21]:
# LM
loss_LM, std_LM = [], []
layer_numbers = [5, 10, 50, 60, 70, 80, 100]
budgets=np.linspace(-100, 100, 20)
for N in layer_numbers:
    cur_loss, cur_std = make_pert_LM(N, budgets, rep_num=10, show=True)
    loss_LM.append(cur_loss)
    std_LM.append(cur_std)

dim 5, rep 0
dim 5, rep 1
dim 5, rep 2
dim 5, rep 3
dim 5, rep 4
dim 5, rep 5
dim 5, rep 6
dim 5, rep 7
dim 5, rep 8
dim 5, rep 9
dim 10, rep 0
dim 10, rep 1
dim 10, rep 2
dim 10, rep 3
dim 10, rep 4
dim 10, rep 5
dim 10, rep 6
dim 10, rep 7


KeyboardInterrupt: 

In [27]:
def plot(loss, std):
    fig, ax = plt.subplots(1, 1)
    for cur_loss, cur_std, layer_number, i in zip(loss, std, layer_numbers, range(len(std))):
        ax.plot(budgets, cur_loss, label=f'{layer_number} layers', c=f'C{i}')
        ax.fill_between(budgets, cur_loss - cur_std, cur_loss + cur_std, color=f'C{i}', alpha=0.1)

    ax.set_xlabel('pert')
    ax.set_ylabel('loss after gd')
    ax.set_yscale('log')
    ax.legend()

plot(loss_adam, std_adam)
# plot(loss_LM, std_LM)

In [25]:
ax.set_ylim(0, None)


(0.0, 0.1677856300692068)